In [ ]:
%reload_ext autoreload
%autoreload 2
#show all columnsdf
#ignore warnings.

In [11]:
import pandas as pd
pd.set_option('display.max_columns', 9000)
import numpy as np
import os, sys
import importlib
import yaml, json
from pathlib import Path
inst=Path('../Data/institutions/')
csc=Path('../Data/College_Scorecard_Raw_Data_01162025/')


In [12]:
df=pd.read_csv(inst/'institutions.csv')
cs=pd.read_csv(csc/'MERGED2022_23_PP.csv')
df

/var/folders/bv/2rb8dc7100ncxqhclz10zf5c0000gn/T/ipykernel_33279/4051684229.py:2: DtypeWarning: Columns (9,1725,1726,1727,1728,1729,1743,1815,1816,1817,1818,1823,1824,1830,1831,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1909,1910,1911,1912,1913,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1983,1984,2403,2404,2495,2496,2497,2498,2499,2500,2501,2502,2503,2504,2505,2506,2507,2508,2509,2510,2511,2512,2513,2514,2515,2516,2517,2518,2519,2520,2521,2522,2523,2524,2525,2526,2527,2528,2529,2530,3215,3231,3235,3236) have mixed types. Specify dtype option on import or set low_memory=False.
  cs=pd.read_csv(csc/'MERGED2022_23_PP.csv')


,Institution_OrgUEINum_C,Sum,Count,Institution_Name_C
0,GNJ7BBP73WE9,2.013853e+09,4689,Regents of the University of Michigan - Ann Arbor
1,HD1WMN6945W6,2.095012e+09,4214,University of Washington
2,Y8CWNJRCNN91,2.623375e+09,4069,University of Illinois at Urbana-Champaign
3,NPM2J7MSCF61,1.487659e+09,3861,Pennsylvania State Univ University Park
4,GS3YEVSS12N6,1.879913e+09,3758,University of California-Berkeley
...,...,...,...,...
17400,Butler Annie,0.000000e+00,0,Butler Annie
17401,Ray Emily A,0.000000e+00,0,Ray Emily A
17402,Herbold Craig W,0.000000e+00,0,Herbold Craig W
17403,Swedish Health Services,0.000000e+00,0,Swedish Health Services


In [21]:
from fuzzywuzzy import process
import pandas as pd

# Function to find the closest match and get corresponding details from the cs DataFrame
def find_closest_match(name, choices, cs_df):
    match = process.extractOne(name, choices)
    # Get the corresponding row from the cs DataFrame based on the closest match
    matched_row = cs_df[cs_df['INSTNM'] == match[0]].iloc[0]
    return match[0], matched_row['UNITID'], matched_row['OPEID'], matched_row['OPEID6']

# Initialize an empty list to store results
results = []
batch_size = 100  # Save every 100 records

for i, (name, org_uei) in enumerate(zip(df['Institution_Name_C'], df['Institution_OrgUEINum_C']), start=1):
    # Append the result along with Institution_OrgUEINum_C to the list
    match_result = find_closest_match(name, cs['INSTNM'], cs)
    results.append((org_uei, *match_result))
    
    # Save progress every 100 records
    if i % batch_size == 0:
        print(f"Processed {i} records, saving progress to CSV...")
        
        # Convert results to a temporary DataFrame, including Institution_OrgUEINum_C
        temp_df = pd.DataFrame(results, columns=[
            'Institution_OrgUEINum_C', 'Closest_Match', 'UNITID', 'OPEID', 'OPEID6'
        ])
        
        # Save the temporary results to a CSV file
        temp_df.to_csv(f"progress_{i}.csv", index=False)

# Final save for any remaining records
print("Final save...")
final_df = pd.DataFrame(results, columns=[
    'Institution_OrgUEINum_C', 'Closest_Match', 'UNITID', 'OPEID', 'OPEID6'
])
final_df.to_csv("final_results.csv", index=False)

# Add results to the original DataFrame
df[['Closest_Match', 'UNITID', 'OPEID', 'OPEID6']] = final_df[
    ['Closest_Match', 'UNITID', 'OPEID', 'OPEID6']
]

Processed 100 records, saving progress to CSV...
Processed 200 records, saving progress to CSV...
Processed 300 records, saving progress to CSV...
Processed 400 records, saving progress to CSV...
Processed 500 records, saving progress to CSV...
Processed 600 records, saving progress to CSV...
Processed 700 records, saving progress to CSV...
Processed 800 records, saving progress to CSV...
Processed 900 records, saving progress to CSV...
Processed 1000 records, saving progress to CSV...
Processed 1100 records, saving progress to CSV...
Processed 1200 records, saving progress to CSV...
Processed 1300 records, saving progress to CSV...
Processed 1400 records, saving progress to CSV...
Processed 1500 records, saving progress to CSV...
Processed 1600 records, saving progress to CSV...
Processed 1700 records, saving progress to CSV...
Processed 1800 records, saving progress to CSV...
Processed 1900 records, saving progress to CSV...
Processed 2000 records, saving progress to CSV...
Processed

In [24]:
df.to_csv("../Data/institutions/nearest_with_matches.csv", index=False)

In [25]:
df

,Institution_OrgUEINum_C,Sum,Count,Institution_Name_C,Closest_Match,UNITID,OPEID,OPEID6
0,GNJ7BBP73WE9,2.013853e+09,4689,Regents of the University of Michigan - Ann Arbor,Alabama A & M University,100654,100200.0,1002.0
1,HD1WMN6945W6,2.095012e+09,4214,University of Washington,University of Mary Washington,232681,374600.0,3746.0
2,Y8CWNJRCNN91,2.623375e+09,4069,University of Illinois at Urbana-Champaign,University of Illinois Urbana-Champaign,145637,177500.0,1775.0
3,NPM2J7MSCF61,1.487659e+09,3861,Pennsylvania State Univ University Park,Alabama A & M University,100654,100200.0,1002.0
4,GS3YEVSS12N6,1.879913e+09,3758,University of California-Berkeley,University of California-Berkeley,110635,131200.0,1312.0
...,...,...,...,...,...,...,...,...
17400,Butler Annie,0.000000e+00,0,Butler Annie,Butler Community College,154800,190600.0,1906.0
17401,Ray Emily A,0.000000e+00,0,Ray Emily A,Alabama A & M University,100654,100200.0,1002.0
17402,Herbold Craig W,0.000000e+00,0,Herbold Craig W,Fred W Eberle Technical Center,237844,2534800.0,25348.0
17403,Swedish Health Services,0.000000e+00,0,Swedish Health Services,Academy of Chinese Culture and Health Sciences,108269,3288300.0,32883.0


In [20]:
from fuzzywuzzy import process
import pandas as pd

# Function to find the closest match and get corresponding details from the cs DataFrame
def find_closest_match(name, choices, cs_df):
    match = process.extractOne(name, choices)
    # Get the corresponding row from the cs DataFrame based on the closest match
    matched_row = cs_df[cs_df['INSTNM'] == match[0]].iloc[0]
    return match[0], matched_row['UNITID'], matched_row['OPEID'], matched_row['OPEID6']

# Initialize an empty DataFrame to store results
results = []
batch_size = 100  # Save every 100 records

for i, name in enumerate(df['Institution_Name_C'], start=1):
    # Append the result to the list
    results.append(find_closest_match(name, cs['INSTNM'], cs))
    
    # Save progress every 100 records
    if i % batch_size == 0:
        print(f"Processed {i} records, saving progress to CSV...")
        
        # Convert results to a temporary DataFrame
        temp_df = pd.DataFrame(results, columns=['Closest_Match', 'UNITID', 'OPEID', 'OPEID6'])
        
        # Save the temporary results to a CSV file
        temp_df.to_csv(f"../temp/progress.csv", index=False)

# Final save for any remaining records
print("Final save...")
final_df = pd.DataFrame(results, columns=['Closest_Match', 'UNITID', 'OPEID', 'OPEID6'])
final_df.to_csv("../Data/institutions/instituions_closest.csv", index=False)

# Add results to the original DataFrame
df[['Closest_Match', 'UNITID', 'OPEID', 'OPEID6']] = final_df

KeyboardInterrupt: 

In [9]:
r1

,Institution_OrgUEINum,Institution_Name,Research
0,H4VNDUN2VWU5,american university,R2
1,NTLHJXM55KZ6,arizona state university,R1
2,DMQNDJDHTDG4,auburn university,R1
3,TNQMP46NDSL1,azusa pacific university,R2
4,KDP6QKY6QLM1,ball state university,R2
...,...,...,...
169,MNL4D3VQUWA5,west virginia university,R1
170,J7WULLYGFRH1,western michigan university,R2
171,JKKNZLNYLJ19,wichita state university,R2
172,HJNQME41NBU4,worcester polytechnic institute,R2


In [8]:
#merge df with R1
pd.merge(df, r1, left_on='Institution_OrgUEINum_C', right_on='Institution_OrgUEINum', how='left')

,Institution_OrgUEINum_C,Sum,Count,Institution_Name_C,Institution_OrgUEINum,Institution_Name,Research
0,GNJ7BBP73WE9,2.013853e+09,4689,Regents of the University of Michigan - Ann Arbor,NaN,NaN,NaN
1,HD1WMN6945W6,2.095012e+09,4214,University of Washington,NaN,NaN,NaN
2,Y8CWNJRCNN91,2.623375e+09,4069,University of Illinois at Urbana-Champaign,NaN,NaN,NaN
3,NPM2J7MSCF61,1.487659e+09,3861,Pennsylvania State Univ University Park,NaN,NaN,NaN
4,GS3YEVSS12N6,1.879913e+09,3758,University of California-Berkeley,GS3YEVSS12N6,university of california-berkeley,R1
...,...,...,...,...,...,...,...
17400,Butler Annie,0.000000e+00,0,Butler Annie,NaN,NaN,NaN
17401,Ray Emily A,0.000000e+00,0,Ray Emily A,NaN,NaN,NaN
17402,Herbold Craig W,0.000000e+00,0,Herbold Craig W,NaN,NaN,NaN
17403,Swedish Health Services,0.000000e+00,0,Swedish Health Services,NaN,NaN,NaN
